In [14]:
import inspect
from typing import Union
from pandas import DataFrame as DF
import re
from os import PathLike

import pandas
import numpy as np
from typing import TypeVar, Callable

T = TypeVar("T")
def singleton(self: T, method: Callable[[T, ...], any], *args, **kwargs) -> T:
    method(*args, **kwargs)
    return self

In [15]:

column_names = ['noteid', 'de_word', 'de_word_audio', 'de_word_alt', 'de_word_note', 'de_word_article', 'de_word_plural', 'de_word_plural_audio', 'de_word_inflected', 'de_word_inflected_audio',
                'en_word_1', 'en_word_1_note', 'de_sen_1', 'de_sen_1_audio', 'en_sen_1', 'de_sen_1_note',
                'en_word_2', 'en_word_2_note', 'de_sen_2', 'de_sen_2_audio', 'en_sen_2', 'de_sen_2_note',
                'en_word_3', 'en_word_3_note', 'de_sen_3', 'de_sen_3_audio', 'en_sen_3', 'de_sen_3_note',
                'en_word_4', 'en_word_4_note', 'de_sen_4', 'de_sen_4_audio', 'en_sen_4', 'de_sen_4_note',
                'en_word_5', 'en_word_5_note', 'de_sen_5', 'de_sen_5_audio', 'en_sen_5', 'de_sen_5_note',
                'en_word_6', 'en_word_6_note', 'de_sen_6', 'de_sen_6_audio', 'en_sen_6', 'de_sen_6_note',
                'en_word_7', 'en_word_7_note', 'de_sen_7', 'de_sen_7_audio', 'en_sen_7', 'de_sen_7_note',
                'en_word_8', 'en_word_8_note', 'de_sen_8', 'de_sen_8_audio', 'en_sen_8', 'de_sen_8_note',
                'en_word_9', 'en_word_9_note', 'de_sen_9', 'de_sen_9_audio', 'en_sen_9', 'de_sen_9_note',
                'en_word_10', 'en_word_10_note', 'de_sen_10', 'de_sen_10_audio', 'en_sen_10', 'fullgerman', 'part_of_speech', 'level', 'tags']
FilePath = Union[str, "PathLike[str]"]


def loadAnki(path: FilePath) -> DF:
    df: pandas.DataFrame = pandas.read_csv(path, header=None, encoding='utf8', engine='python', dtype=str)
    df.columns = column_names
    df = df.fillna(value='')

    for cn in df.columns:
        mask = df[cn].str.startswith('"')
        df.loc[mask, cn] = [v[1:-1].replace('""','"') for v in df.loc[mask, cn]]

    return df

b1 = loadAnki("./data/DE_ Goethe Institute B1 Wordlist by sirati.csv")
a2 = loadAnki("./data/DE_ Goethe Institute A2 Wordlist by sirati.csv")
a1 = loadAnki("./data/DE_ Goethe Institute A1 Wordlist by sirati.csv")

In [16]:
from typing import Set


def splitDoubleIntoAlt(deck):
    mask = deck.de_word.str.contains('/')
    deck.de_word_alt[mask] = [(word.split('/', 1)[1]+ " " + alt).strip()  for (word, alt) in deck.loc[mask, ['de_word', 'de_word_alt']].values]
    deck.de_word[mask] = [v.split('/', 1)[0].strip()  for v in deck.de_word[mask]]


splitDoubleIntoAlt(a1)
splitDoubleIntoAlt(a2)

maskNotVerb = b1.de_word_inflected.str.strip().str.len() == 0
maskNotNoun = b1.de_word_plural.str.strip().str.len() == 0
maskHasAlternative = b1.fullgerman.str.contains('/')

#a1.de_word_alt[mask] = [(word.split('/', 1)[1]+ " " + alt).strip()  for (word, alt) in a1.loc[mask, ['de_word', 'de_word_alt']].values]
#a1.de_word[mask] = [v.split('/', 1)[0].strip()  for v in a1.de_word[mask]]
#mask


Move non special cases to word_alt

In [17]:
maskSpecialEasy = maskNotVerb & maskHasAlternative & maskNotNoun

b1Alt: DF = b1.loc[maskSpecialEasy, ['de_word', 'de_word_alt', 'de_word_inflected','de_word_plural', 'fullgerman']]
b1Alt.fullgerman = b1Alt.fullgerman.str.lower().str.split('/').apply(set)

b1Alt.fullgerman = [singleton(s, s.discard, word.lower()) for (word, s) in b1Alt.loc[:, ['de_word', 'fullgerman']].values]
b1.de_word_alt[maskSpecialEasy] = [('/'.join(s) + " " + alt).strip()  for (alt, s) in b1Alt.loc[:, ['de_word_alt', 'fullgerman']].values]

Ensure that double nouns have correct plural

In [18]:
maskSpecialNoun = maskNotVerb & maskHasAlternative & ~maskNotNoun


b1Alt: DF = b1.loc[maskSpecialNoun, ['de_word', 'de_word_alt', 'de_word_article','de_word_plural', 'fullgerman']].copy()
type1 = b1Alt.fullgerman.str.contains(r'^(d(er|as|ie) +\w+[^\/\n\r]+(\/|(?=$)) *)+$') #python no atomic groups :( ^(?>d(er|as|ie) +\w+[^\/\n\r]+(\/|(?=$)) *)+$
b1type1 = b1Alt[type1].copy()
type2 = b1Alt.fullgerman.str.contains(r'\/[^\/\n]+,[^\/\n]+\/')
b1type2 = b1Alt[type2].copy()
b1type2.loc[:,'singular'] = b1type2.fullgerman.str.lower().str.split(r',(?=[^\/\n]+\/)')
b1type2.loc[:,'ending'] = [list(map(str.strip, l[1].strip().split('/'))) for l in b1type2.singular]
b1type2.singular = [list(map(str.strip, l[0].strip().split('/'))) for l in b1type2.singular]

b1type1.loc[:,'singular'] = b1type1.fullgerman.str.lower().str.split('/')
b1type1.singular =[[s.strip().split(',') for s in l] for l in b1type1.singular]
b1type1.loc[:,'ending'] = [[s[1].strip() if len(s)>1 else None for s in l] for l in b1type1.singular]
b1type1.singular = [[s[0].strip() for s in l] for l in b1type1.singular]

b1AltNoun = pandas.concat([b1type1, b1type2])
b1AltNoun.loc[:,'article'] = [[s[:3] if s.startswith('der') or s.startswith('die') or s.startswith('das') else None for s in l] for l in b1AltNoun.singular]
b1AltNoun.singular =[[re.sub(r'^d(er|as|ie) +', '', s) for s in l] for l in b1AltNoun.singular]


b1AltNoun.loc[:,'has_umlaut'] = [['"' in s if s is not None else False for s in l] for l in b1AltNoun.ending]
b1AltNoun.ending = [[s.replace('"','').replace('-','') if s is not None else None for s in l] for l in b1AltNoun.ending]
b1AltNoun.loc[:,'first_aou'] = [[max(s.rfind('o'),
                                     s.rfind('a'),
                                     (lambda au, u: au if au+1==u else max(au, u))(
                                         s.rfind('au'),
                                         s.rfind('u')))
                                 for s in l] for l in b1AltNoun.singular]

b1AltNoun.loc[:,'plural'] = [set([None if ending is None else
                              ((singular[:first_aou] +
                                singular[first_aou].replace('a','ä')
                                .replace('o','ö')
                                .replace('u','ü') +
                                singular[first_aou+1:]
                                if has_umlaut else singular) + ending).title() for (singular, ending, has_umlaut, first_aou) in zip(*s)]) for s in b1AltNoun.loc[:,['singular','ending','has_umlaut','first_aou']].values]

#b1AltNoun.singular =[filter(word.lower(),set(s)) for (s, word) in b1AltNoun.loc[:,['singular','de_word']].values]
b1AltNoun.singular =[set(s) for s in b1AltNoun.singular]
b1AltNoun.singular = [singleton(s, s.discard, word.lower()) for (word, s) in b1AltNoun.loc[:, ['de_word', 'singular']].values]
b1AltNoun.singular = [[s.title() for s in l] for l in b1AltNoun.singular]

#b1AltNoun.apply(singleton(s, s.discard, word.lower()))
b1AltNoun.de_word_plural = b1AltNoun.plural.apply(lambda s: ", ".join(filter(None,s)))
b1AltNoun.de_word_article = b1AltNoun.article.apply(lambda s: ", ".join(filter(None,s)))
b1AltNoun.de_word_alt = b1AltNoun.singular.apply(lambda s: ", ".join(filter(None,s)))


b1.loc[maskSpecialNoun & (type1 | type2), ['de_word_alt', 'de_word_article','de_word_plural']] = b1AltNoun.loc[:, ['de_word_alt', 'de_word_article','de_word_plural']]
b1[maskNotVerb & maskHasAlternative]

C:\Users\sirati97\AppData\Local\Temp/ipykernel_924/3107278971.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  type1 = b1Alt.fullgerman.str.contains(r'^(d(er|as|ie) +\w+[^\/\n\r]+(\/|(?=$)) *)+$') #python no atomic groups :( ^(?>d(er|as|ie) +\w+[^\/\n\r]+(\/|(?=$)) *)+$


,noteid,de_word,de_word_audio,de_word_alt,de_word_note,de_word_article,de_word_plural,de_word_plural_audio,de_word_inflected,de_word_inflected_audio,...,de_sen_9_note,en_word_10,en_word_10_note,de_sen_10,de_sen_10_audio,en_sen_10,fullgerman,part_of_speech,level,tags
225,1516347476435,Bank,[sound:sapi5js-90b96939-174ef917-fe2bae9e-94a7...,,,"die, die","Banken, Bänke",,,,...,,,,,,,"die Bank, -""e/die Bank, -en",,226,
433,1516347476643,Cousin,[sound:sapi5js-a594c371-8609ad86-2b272396-90a6...,Cousine,,"der, die","Cousinen, Cousins",,,,...,,,,,,,"der Cousin, -s / die Cousine, -n",,434,
482,1516347476692,Diskothek,[sound:sapi5js-ea6c3d9f-bc74154c-6310c4f3-59bd...,Disko,,"die, die","Diskotheken, Diskos",,,,...,,,,,,,"die Diskothek, -en / die Disko, -s",,483,
524,1516347476734,Ehefrau,[sound:sapi5js-ac1d1fca-bd0a7d05-1f848ee0-3049...,Ehemann,,"die, der","Ehefrauen, Ehemänner",,,,...,,,,,,,"die Ehefrau, -en / der Ehemann, -""er",,525,
525,1516347476735,Ehegatte,[sound:sapi5js-0f7c2800-09da8220-d89e5a8c-926d...,Ehegattin,,"der, die","Ehegattinnen, Ehegatten",,,,...,,,,,,,"der Ehegatte, -n / die Ehegattin, -nen",,526,
668,1516347476878,erschrecken,[sound:sapi5js-3b80dce8-55bbe312-64fb6076-83ac...,"jdn. erschrecken, erschreckt, erschreckte/ersc...",,,,,,,...,,,,,,,"erschrecken, erschrickt, erschrak, ist erschro...",,669,
816,1516347477026,früher,[sound:sapi5js-64dab7f7-c5a76c41-c99fe338-deec...,früher-,,,,,,,...,,,,,,,früher/früher-,,817,
893,1516347477103,gerne,[sound:sapi5js-fc47902f-4b3dc2b7-8742ef8a-311a...,gern,,,,,,,...,,,,,,,gern/gerne,,894,
894,1516347477104,gesamte,[sound:sapi5js-316fad4f-5dd4a023-d740e6cd-2600...,gesamt-,,,,,,,...,,,,,,,gesamt-/Gesamt-,,895,
994,1516347477204,hängen,[sound:sapi5js-208f71f2-5eee5c4b-6d331c97-d3b1...,"hängen, hängt, hängte, hat gehängt/hängen, hän...",,,,,,,...,,,,,,,"hängen, hängt, hing, hat gehangen / hängen, hä...",,995,


In [19]:
def fixSingularMark(deck):
    mask = deck.de_word_plural == "Sg."
    deck.de_word_plural[mask] = ""
    deck.de_word_note[mask] = "Sg."


fixSingularMark(a1)
fixSingularMark(a2)
fixSingularMark(b1)

b1[~maskNotNoun]

,noteid,de_word,de_word_audio,de_word_alt,de_word_note,de_word_article,de_word_plural,de_word_plural_audio,de_word_inflected,de_word_inflected_audio,...,de_sen_9_note,en_word_10,en_word_10_note,de_sen_10,de_sen_10_audio,en_sen_10,fullgerman,part_of_speech,level,tags
2,1516347476212,Abbildung,[sound:sapi5js-3075b7a4-967483d5-18ecf061-736c...,,,die,Abbildungen,,,,...,,,,,,,"die Abbildung, -en",,3,
4,1516347476214,Abfall,[sound:sapi5js-3a56fed8-b6c53645-a125b433-6c6d...,,,der,Abfälle,,,,...,,,,,,,"der Abfall, -""e",,5,
6,1516347476216,Abgas,[sound:sapi5js-8bbe9376-08f6cc1a-06f31bcf-6f4b...,,,das,Abgase,,,,...,,,,,,,"das Abgas, -e",,7,
12,1516347476222,Abitur,[sound:sapi5js-bc5772bd-73dba63f-08fba3d3-ad66...,,Sg.,das,,,,,...,,,,,,,das Abitur (Sg.),,13,
17,1516347476227,Abonnement,[sound:sapi5js-1037e91b-74cb1731-c4ca58ca-fca4...,,,das,Abonnements,,,,...,,,,,,,"das Abonnement, -s",,18,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,1516347478831,Zustand,[sound:sapi5js-93a77598-a21dbf74-e236ef12-2f19...,,,der,Zustände,,,,...,,,,,,,"der Zustand, -""e",,2621,
2623,1516347478833,Zustimmung,[sound:sapi5js-eb2825a6-e94e85b2-7f1f486a-c537...,,,die,Zustimmungen,,,,...,,,,,,,"die Zustimmung, -en",,2623,
2625,1516347478835,Zuwanderer,[sound:sapi5js-63026f71-c5fa0ab4-d3b4403c-b5f6...,,,der,Zuwanderer,,,,...,,,,,,,"der Zuwanderer, -",,2625,
2627,1516347478837,Zweck,[sound:sapi5js-d3569cc1-6a4314a3-200cfdcf-6dbe...,,,der,Zwecke,,,,...,,,,,,,"der Zweck, -e",,2627,


In [20]:
def fixPartOfSpeech(deck):
    maskVerb = deck.de_word_inflected.str.strip().str.len() > 0
    maskNoun = deck.de_word_plural.str.strip().str.len() > 0
    maskMissingPoS = deck.part_of_speech.str.strip().str.len() == 0
    maskPoSInflected = deck.part_of_speech.str.strip().str.lower() == "inflected"
    deck.part_of_speech[maskVerb & maskMissingPoS] = "verb" #inflected might be adj so cannot set to verb
    deck.part_of_speech[maskNoun & (maskMissingPoS|maskPoSInflected)] = "noun"


fixPartOfSpeech(a1)
fixPartOfSpeech(a2)
fixPartOfSpeech(b1)

In [21]:
def GenVocSet(deck):
    vocab = pandas.concat([deck.de_word, deck.de_word_alt])
    vocab = vocab.str.lower().str.replace("-|\(sich\) +", '', regex=True)
    mask = vocab.str.contains("/")
    vocab[mask] = vocab[mask].str.split("/")
    vocab = pandas.Series(np.hstack(vocab))
    vocab = vocab.str.replace("^([^\(]*)\(([^\)]+)\)(.*)$", r'\1\3/\1\2\3', regex=True).str.split("/")
    return set(np.hstack(vocab))

a1Vocab = GenVocSet(a1)
a2Vocab = GenVocSet(a2)
b1Vocab = GenVocSet(b1)
(set(a1Vocab) & set(b1Vocab))

{'',
 'ansage',
 'anschluss',
 'antwort',
 'antworten',
 'anzeige',
 'anziehen',
 'apartment',
 'apfel',
 'appetit',
 'arbeit',
 'arbeiten',
 'arbeitslos',
 'arbeitsplatz',
 'arm',
 'arzt',
 'auch',
 'auf',
 'auf sein',
 'aufgabe',
 'aufhören',
 'aufstehen',
 'aufzug',
 'auge',
 'aus',
 'aus sein',
 'ausflug',
 'ausfüllen',
 'ausgang',
 'auskunft',
 'ausland',
 'ausländer',
 'ausländisch',
 'ausmachen',
 'aussehen',
 'ausweis',
 'ausziehen',
 'auto',
 'autobahn',
 'automat',
 'automatisch',
 'baby',
 'bad',
 'baden',
 'bahn',
 'bahnhof',
 'bahnsteig',
 'bald',
 'balkon',
 'banane',
 'bank',
 'bar',
 'bauch',
 'baum',
 'beamte',
 'bedeuten',
 'beginnen',
 'bei',
 'beide',
 'bein',
 'beispiel',
 'bekannt',
 'bekannte',
 'bekommen',
 'benutzen',
 'beruf',
 'besichtigen',
 'bestellen',
 'besuchen',
 'bett',
 'bezahlen',
 'bier',
 'bild',
 'billig',
 'birne',
 'bis',
 'bitte',
 'bitten',
 'bitter',
 'blau',
 'bleiben',
 'bleistift',
 'blick',
 'blume',
 'bogen',
 'brauchen',
 'braun',
 'bre

In [22]:
def markDup(deck, others: dict[str, set[str]]):
    for label, s in others.items():
        mask = np.vectorize(s.__contains__)(deck.de_word.str.lower())
        deck.tags[mask] += " " + label


markDup(a2, {'a1': a1Vocab})
markDup(b1, {'a1': a1Vocab, 'a2': a2Vocab})



In [23]:
a1.to_csv("./data/DE_ Goethe Institute A1 Wordlist by siratiOut.csv", header=False, index=False, encoding='utf8')
a2.to_csv("./data/DE_ Goethe Institute A2 Wordlist by siratiOut.csv", header=False, index=False, encoding='utf8')
b1.to_csv("./data/DE_ Goethe Institute B1 Wordlist by siratiOut.csv", header=False, index=False, encoding='utf8')